<div style="max-width: 600px; margin: 20px auto 22px; padding: 0px; border-radius: 18px; border: 1px solid #e5e7eb; background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%); box-shadow: 0 8px 26px rgba(0,0,0,0.06); overflow: hidden;">

  <!-- Banner Header -->
  <div style="padding: 34px 32px 14px; text-align: center; line-height: 1.38;">
    <div style="font-size: 13px; letter-spacing: 0.14em; text-transform: uppercase; color: #6b7280; font-weight: bold; margin-bottom: 5px;">
      Session #1
    </div>
    <div style="font-size: 29px; font-weight: 800; color: #14276c; margin-bottom: 4px;">
      Cleaning Time Series Data
    </div>
    <div style="font-size: 16.5px; color: #374151; font-style: italic; margin-bottom: 0;">
      Advanced Training School: Methods for Time Series
    </div>
  </div>

  <!-- Logo Section -->
  <div style="background: none; text-align: center; margin: 30px 0 10px;">
    <img src="https://www.cemfi.es/images/Logo-Azul.png" alt="CEMFI Logo" style="width: 158px; filter: drop-shadow(0 2px 12px rgba(56,84,156,0.05)); margin-bottom: 0;">
  </div>

  <!-- Name -->
  <div style="font-family: 'Times New Roman', Times, serif; color: #38549c; text-align: center; font-size: 1.22em; font-weight: bold; margin-bottom: 0px;">
    Jesus Villota Miranda © 2025
  </div>

  <!-- Contact info -->
  <div style="font-family: 'Times New Roman', Times, serif; color: #38549c; text-align: center; font-size: 1em; margin-top: 7px; margin-bottom: 20px;">
    <a href="mailto:jesus.villota@cemfi.edu.es" style="color: #38549c; text-decoration: none; margin-right:8px;" title="Email">
      jesus.villota@cemfi.edu.es
    </a>
    <span style="color:#9fa7bd;">|</span>
    <a href="https://www.linkedin.com/in/jesusvillotamiranda/" target="_blank" style="color: #38549c; text-decoration: none; margin-left:7px;" title="LinkedIn">
      LinkedIn
    </a>
  </div>
</div>


## Learning Objectives

1. Load and clean raw financial and economic time series data from FRED
2. Convert dates to proper Stata time series formats
3. Create standard transformations (logs, returns, growth rates)
4. Handle missing values and data quality issues
5. Prepare cleaned datasets ready for time series analysis

<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">1) Introduction</h2>

This notebook processes raw time series data downloaded from FRED (Federal Reserve Economic Data) and prepares it for analysis in subsequent sessions.

**Data Sources:**
- Raw CSV files are stored in `data/raw/`
- Processed Stata `.dta` files will be saved to `data/processed/`
- All data is publicly available from FRED

**Processing Steps:**
1. Load raw CSV files
2. Parse and convert dates to Stata date formats
3. Create standard transformations (logs, returns, growth rates)
4. Handle missing values appropriately
5. Set up time series structure
6. Add variable labels
7. Save processed datasets


<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">2) Setup</h2>


## Why Data Cleaning Matters in Time Series Analysis

Before diving into the code, it's important to understand why data cleaning is particularly critical for time series analysis:

**Time Series-Specific Challenges:**
1. **Temporal Dependencies**: Unlike cross-sectional data, time series observations are ordered and dependent. Missing or incorrectly ordered data can break these dependencies.
2. **Frequency Consistency**: Financial and economic data come at different frequencies (daily, monthly, quarterly). Misaligned frequencies can lead to spurious relationships.
3. **Stationarity Requirements**: Many time series models require stationary data (constant mean, variance). Proper transformations (logs, differences) are essential.
4. **Date Handling**: Time series models rely on proper temporal indexing. Incorrect date formats can invalidate all subsequent analysis.

**Common Issues We'll Address:**
- Missing values (weekends, holidays in financial data)
- Date format standardization (FRED uses YYYY-MM-DD)
- Non-stationarity (we'll create log returns and first differences)
- Mixed frequencies (combining quarterly GDP with monthly CPI)
- Variable scaling (creating percentage returns, growth rates)

**Transformations We'll Apply:**
- **Log transformation**: For prices and levels, $\ln(P_t)$, to stabilize variance and interpret changes as percentage changes
- **First differences**: $\Delta x_t = x_t - x_{t-1}$, to achieve stationarity
- **Returns**: $r_t = \ln(P_t/P_{t-1})$, the standard measure in finance

This notebook will prepare three datasets for subsequent time series analysis:
1. **S&P 500**: Daily stock returns for ARMA and GARCH models
2. **EUR/USD**: Daily exchange rates for unit root testing
3. **Macro-Finance**: Monthly economic indicators for VAR analysis

## Understanding the Setup Commands

Before we begin processing data, we need to prepare our Stata environment. Each setup command serves a specific purpose:

### The `clear all` Command

**Syntax:** `clear all`

**What it does:** Removes everything from Stata's memory:
- All data currently loaded
- All value labels
- All matrices
- All scalars and macros
- Closes any open graph windows

**Why we use it:** 
- Ensures a clean slate - no leftover data from previous sessions
- Prevents variable name conflicts
- Essential for reproducibility - ensures the script runs the same way every time
- Best practice: always start analysis scripts with `clear all`

**Alternative:** `clear` (only clears data, not everything)

---

### The `set more off` Command

**Syntax:** `set more off`

**What it does:** Disables Stata's pagination feature

**Why we use it:**
- By default, Stata pauses output when the Results window is full, showing `--more--`
- In scripts and notebooks, we want continuous output without manual intervention
- Makes logs and output easier to read
- Standard practice for automated workflows

**Alternative:** `set more on` (re-enables pagination, useful for interactive sessions)

**Note:** In modern Stata versions and notebooks, this is less critical but still good practice.

---

### The `cd` Command (Change Directory)

**Syntax:** `cd "path/to/directory"`

**What it does:** Changes the current working directory

**Why we use it:**
- All relative file paths are interpreted from the working directory
- `cd "../.."` moves up two levels from `session_1/notebooks/` to the project root
- Ensures we can access `data/raw/` and save to `data/processed/`

**Path Navigation:**
- `..` means parent directory (one level up)
- `../..` means two levels up
- `/` for absolute paths (from root)
- `./` means current directory (optional)

**Best Practice:** 
- Use relative paths from a consistent project root
- Makes code portable across different computers
- Avoids hardcoded absolute paths like `/Users/yourname/...`

**Checking your location:** Use `pwd` (print working directory) to see where you are

---

### Why This Order Matters

1. **First**: `clear all` - clean the workspace
2. **Second**: `set more off` - configure output behavior  
3. **Third**: `cd` - set location for file operations

This ensures we start fresh, configure our environment, then position ourselves correctly for data access.

In [2]:
* Setup
clear all
set more off
* Set working directory to project root (from notebooks folder)
cd "../.."





/Users/jesusvillotamiranda/Library/CloudStorage/OneDrive-UniversidaddeLaRioja/Gi
> tHub/Repository/TA_Time_Series_Methods
> tHub/Repository/TA_Time_Series_Methods


<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">3) S&P 500 Index Data</h2>

Process daily S&P 500 index data. This will be used for ARMA modeling and volatility analysis.


### Step 1: Importing CSV Data and Initial Inspection

Before we can analyze data, we need to load it into Stata's memory. The S&P 500 data comes from FRED (Federal Reserve Economic Data) as a CSV file.

**The `import delimited` Command**

**Full Syntax:**
```stata
import delimited [using] filename [, options]
```

**Key Options:**
- `clear`: Replace data currently in memory (required if data already loaded)
- `delimiter("char")`: Specify delimiter (default is comma for .csv)
- `varnames(#)`: Row number containing variable names (default = 1)
- `stringcols(#)`: Force specific columns to be strings
- `numericcols(#)`: Force specific columns to be numeric
- `case(preserve|lower|upper)`: How to handle variable name case

**Our Command:**
```stata
import delimited "data/raw/SP500.csv", clear
```

**What it does:**
- Loads the CSV file from the `data/raw/` folder
- `clear`: Replaces any existing data in memory
- Automatically detects comma as delimiter (it's a .csv file)
- Reads first row as variable names
- Infers data types (string vs numeric) from the first few rows

---

**The `list` Command**

**Syntax:** `list [varlist] [if] [in] [, options]`

**Our use:** `list in 1/10`
- `in 1/10`: Shows observations 1 through 10
- Useful for quick preview of data structure
- Alternative: `list in 1/5` (first 5), `list in -10/-1` (last 10)

---

**The `describe` Command**

**Syntax:** `describe [varlist] [, options]`

**What it shows:**
- Number of observations and variables
- Variable names, storage types (byte, int, long, float, double, str#)
- Display format
- Value labels (if any)
- Variable labels (if any)

**Why this matters for time series:**
- Confirms we have the expected variables (date and price)
- Checks data types (dates often import as strings and need conversion)
- Shows the number of observations (determines sample size for analysis)

**Expected Output:**
- `observation_date`: likely string type (str10 or similar)
- `sp500`: likely float or double (numeric)

In [ ]:
* Load S&P 500 raw data
import delimited "data/raw/SP500.csv", clear

* Display first few observations
list in 1/10

* Check data structure
describe



(encoding automatically selected: ISO-8859-1)
(2 vars, 2,609 obs)


     +----------------------+
     | observat~e     sp500 |
     |----------------------|
  1. | 2015-11-04   2102.31 |
  2. | 2015-11-05   2099.93 |
  3. | 2015-11-06    2099.2 |
  4. | 2015-11-09   2078.58 |
  5. | 2015-11-10   2081.72 |
     |----------------------|
  6. | 2015-11-11      2075 |
  7. | 2015-11-12   2045.97 |
  8. | 2015-11-13   2023.04 |
  9. | 2015-11-16   2053.19 |
 10. | 2015-11-17   2050.44 |
     +----------------------+


Contains data
 Observations:         2,609                  
    Variables:             2                  
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
observation_d~e str10   %10s                  
sp500           float   %9.0g                 SP500
--------

### Step 2: Converting String Dates to Stata Date Format

One of the most critical steps in time series analysis is proper date handling. Stata has a sophisticated date system, but dates from CSV files typically import as strings.

**Understanding Stata's Date System**

Stata stores dates internally as **numbers**:
- Daily dates: days since January 1, 1960
  - January 1, 1960 = 0
  - January 2, 1960 = 1
  - December 31, 1959 = -1
- This allows for easy date arithmetic (e.g., `date + 7` adds a week)

**The Difference Between Storage and Display:**
- **Storage format**: How Stata stores the date internally (always a number)
- **Display format**: How Stata shows the date to you (e.g., "01jan2020", "2020-01-01")
- The `%td`, `%tm`, `%tq` formats are display formats

---

**The `date()` Function**

**Syntax:** `date(string_date, "mask")`

**What it does:** Converts a string containing a date to Stata's numeric date format

**Common Masks:**
- `"DMY"`: Day-Month-Year (e.g., "31-12-2020" or "31/12/2020")
- `"MDY"`: Month-Day-Year (e.g., "12-31-2020" or "12/31/2020")  
- `"YMD"`: Year-Month-Day (e.g., "2020-12-31") ← **FRED uses this**
- `"MY"`: Month-Year for monthly data
- `"Y"`: Year only for annual data

**Our case:**
```stata
gen date_numeric = date(observation_date, "YMD")
```
- FRED uses ISO format: "YYYY-MM-DD" (e.g., "2020-03-15")
- `"YMD"` mask tells Stata to parse year first, then month, then day
- Creates a new variable with the numeric date value
- We keep the original to verify before dropping

**Why we drop `observation_date`:**
- The string version is no longer needed
- Reduces memory usage
- Prevents confusion (one date variable is clearer)

---

**The `format` Command**

**Syntax:** `format varlist %fmt`

**What it does:** Changes how variables are **displayed** (not stored)

**Common Date Formats:**
- `%td`: Daily dates (e.g., "15mar2020")
- `%tw`: Weekly dates  
- `%tm`: Monthly dates (e.g., "2020m3")
- `%tq`: Quarterly dates (e.g., "2020q1")
- `%ty`: Yearly dates

**Our use:**
```stata
format date_numeric %td
```
- Displays the numeric value as a readable date
- Doesn't change the underlying number
- Makes verification easier

---

**The `rename` Command**

**Syntax:** `rename old_varname new_varname`

**Why we rename to `date`:**
- Convention: the time variable should be called `date` (or `time`, `t`)
- Required for `tsset` (we'll see this soon)
- Keeps code consistent across datasets

---

**Handling Duplicates**

**The `duplicates` Commands:**

1. **`duplicates report varlist`**: Shows how many duplicates exist
   - Non-duplicates: observations appearing once
   - Duplicates: observations appearing multiple times
   - Doesn't change data, just reports

2. **`duplicates drop varlist, force`**: Removes duplicate observations
   - Keeps first occurrence of each unique value
   - `force`: Required when dropping based on varlist (not all variables)
   - Critical for time series: each date should appear once

**Why duplicates matter in time series:**
- Time series models assume one observation per time period
- Duplicates can indicate data errors
- `tsset` will fail if duplicates exist

---

**The `sort` Command**

**Syntax:** `sort varlist`

**What it does:** Arranges observations in ascending order

**Why we sort by date:**
- Time series must be in chronological order
- Many time series operations assume sorted data
- Required before `tsset`
- Makes visual inspection easier

**Note:** `sort` is **stable** - observations with the same date value maintain their relative order

In [ ]:
* Parse dates from observation_date column (FRED format: YYYY-MM-DD)
gen date_numeric = date(observation_date, "YMD")
drop observation_date

* Format as Stata daily date
format date_numeric %td
rename date_numeric date

* Check for duplicates
duplicates report date
duplicates drop date, force

* Sort by date
sort date








Duplicates in terms of date

--------------------------------------
   Copies | Observations       Surplus
----------+---------------------------
        1 |         2609             0
--------------------------------------


Duplicates in terms of date

(0 observations are duplicates)



### Step 3: Variable Naming and Missing Value Assessment

Now that we have proper dates, we need to rename our price variable meaningfully and assess data quality.

**The `rename` Command for Data Variables**

**Why descriptive names matter:**
- `sp500` (from CSV) is ambiguous - is it open, close, high, low?
- `sp500_close` clearly indicates this is the closing price
- Future you (and collaborators) will thank you
- Makes code self-documenting

**Naming conventions:**
- Use underscores to separate words: `sp500_close` not `sp500close`
- Start with letters, not numbers
- Avoid Stata reserved words: `if`, `in`, `by`, etc.
- Be consistent: if you use `_close` for one series, use it for all

---

**Understanding Missing Values in Stata**

**Stata's missing value system:**
- Numeric missing: represented by `.` (dot)
- Extended missing: `.a`, `.b`, ..., `.z` (27 types) for different reasons
- String missing: `""` (empty string)

**Critical property:** Missing values are treated as **positive infinity** in comparisons!
- `. > 1000000` evaluates to TRUE
- Always use `missing()` function in conditions
- Never use `if x != .` (this fails!)

---

**The `count` Command**

**Syntax:** `count [if] [in]`

**What it does:** Counts observations meeting a condition

**Our use:**
```stata
count if missing(sp500_close)
```

**The `missing()` function:**
- Returns 1 (true) if value is missing
- Returns 0 (false) if value is not missing
- Works with all missing codes: `.`, `.a`, `.b`, etc.
- **Always use this instead of `== .`**

**Why check for missing values:**
- Missing values are expected in financial data (weekends, holidays)
- But too many might indicate a data problem
- Need to know sample size for analysis
- Some time series commands handle missings automatically, others don't

---

**The `summarize` Command**

**Basic Syntax:** `summarize [varlist] [if] [in] [, detail]`

**Without `detail` option (default):**
- Shows: N, mean, std. dev., min, max
- Fast summary for quick checks

**With `detail` option:**
```stata
summarize sp500_close, detail
```

**Provides comprehensive statistics:**
- **Percentiles:** 1%, 5%, 10%, 25%, 50%, 75%, 90%, 95%, 99%
- **Moments:** 
  - Mean and standard deviation
  - Variance
  - Skewness: measure of asymmetry (0 for symmetric)
  - Kurtosis: measure of tail thickness (3 for normal distribution)
- **Extremes:** Smallest and largest values
- **Sum of weights** and **sum of variable**

**What to look for in S&P 500 data:**
- **Min/Max:** Should be positive (it's a price level)
- **Mean vs Median:** If very different, distribution is skewed
- **Skewness:** Stock prices often right-skewed (occasional large increases)
- **Outliers:** Check if smallest/largest values make sense
- **Sample period:** Inferred from number of observations (roughly 252 trading days/year)

**Why `detail` is important for time series:**
- Helps identify outliers before they affect model estimation
- Distributional properties matter for model assumptions
- Can reveal data entry errors (e.g., negative prices)

In [ ]:
* Rename price variable (column name matches filename: SP500)
rename sp500 sp500_close

* Check for missing values
count if missing(sp500_close)
summarize sp500_close, detail




  95


                            SP500
-------------------------------------------------------------
      Percentiles      Smallest
 1%      1923.67        1829.08
 5%      2081.72        1851.86
10%       2181.9        1852.21       Obs               2,514
25%      2675.81        1853.44       Sum of wgt.       2,514

50%     3509.945                      Mean           3695.084
                        Largest       Std. dev.       1251.14
75%         4470        6851.97
90%      5650.38        6875.16       Variance        1565350
95%      6037.88        6890.59       Skewness       .5599163
99%      6654.72        6890.89       Kurtosis       2.345057


### Step 4: Declaring Time Series Structure with `tsset`

This is one of the **most important commands** in Stata time series analysis. Without `tsset`, you cannot use time series operators.

**The `tsset` Command**

**Syntax:** 
```stata
tsset timevar [, options]              // for single time series
tsset panelvar timevar [, options]     // for panel data
```

**What `tsset` does:**
1. **Declares** which variable contains the time index
2. **Validates** the time series structure:
   - Checks for duplicates (each time point should appear once)
   - Verifies chronological ordering
   - Identifies gaps (missing time periods)
3. **Enables** time series operators: `L.`, `F.`, `D.`, `S.`
4. **Stores** time series settings for the dataset

**Our command:**
```stata
tsset date
```

**Common options:**
- `delta(#)`: Specify time between observations (for irregular data)
- `format(%fmt)`: Override default time format
- `yearly|quarterly|monthly|weekly|daily`: Specify frequency

**What Stata checks:**
- ✓ No duplicate time values
- ✓ Data is sorted by time
- ✓ Identifies gaps (e.g., weekends in daily financial data)

**Output information:**
- Time variable name and type
- Delta: time between observations
- Range: first to last time point  
- Number of periods
- Number of gaps (expected for financial data)

---

**Why `tsset` is Required**

**Time Series Operators Only Work After `tsset`:**

These operators reference other time periods relative to current observation:

1. **Lag operator `L.`**: Previous period
   - `L.varname` = value from t-1
   - `L2.varname` = value from t-2
   - `L3.varname` = value from t-3, etc.

2. **Lead operator `F.`**: Next period
   - `F.varname` = value from t+1
   - `F2.varname` = value from t+2

3. **Difference operator `D.`**: Change from previous period
   - `D.varname` = `varname - L.varname`
   - `D2.varname` = second difference = `D.(D.varname)`

4. **Seasonal difference `S.`**: Change from same season previous year
   - `S12.varname` for monthly data (12 months ago)
   - `S4.varname` for quarterly data (4 quarters ago)

**Without `tsset`, these operators won't work!** Stata won't know what "previous period" means.

---

**Gap Handling**

**Financial data typically has gaps:**
- Stock markets: closed weekends and holidays
- Daily data: 252 trading days/year, not 365
- Stata identifies these automatically
- Gaps don't break time series operators (Stata skips them correctly)

**Example:** If Friday is t=5 and Monday is t=8:
- On Monday: `L.price` correctly references Friday (not Sunday)
- Stata uses the date values, not observation numbers

---

**Panel Data Extension**

For panel data (multiple entities over time):
```stata
tsset country_id year
```
- First variable: cross-section identifier (country, firm, individual)
- Second variable: time identifier
- Allows time series operators within each panel

In [ ]:
* Set time series (required before using D. operator)
tsset date

* Create log price and log returns
* Drop if already exists (in case cell is re-run)
gen log_sp500 = ln(sp500_close)
gen sp500_return = D.log_sp500

* Drop log price (keep only close and return)
drop log_sp500

* Check returns summary
summarize sp500_return, detail




Time variable: date, 04nov2015 to 03nov2025, but with gaps
        Delta: 1 day

(95 missing values generated)

(643 missing values generated)



                        sp500_return
-------------------------------------------------------------
      Percentiles      Smallest
 1%    -.0342684      -.0999451
 5%    -.0172119      -.0616093
10%    -.0111666      -.0607519       Obs               1,966
25%     -.003767      -.0532222       Sum of wgt.       1,966

50%     .0006955                      Mean           .0005213
                        Largest       Std. dev.      .0112914
75%     .0058107        .060544
90%     .0117779       .0888085       Variance       .0001275
95%     .0160189       .0896831       Skewness      -.0679821
99%     .0258198       .0908947       Kurtosis       15.01661


### Step 5: Computing Log Returns - Theory and Practice

Returns are the fundamental building block of financial time series analysis. We need to transform prices into returns for several theoretical and practical reasons.

---

**Why Returns Instead of Prices?**

**Theoretical reasons:**

1. **Stationarity**: Prices typically have trends (non-stationary), returns are closer to stationary
2. **Scale-free**: Returns are comparable across assets regardless of price level
3. **Interpretability**: Returns represent investor gains/losses

**Mathematical definition:**

**Simple returns:**
$$R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \frac{P_t}{P_{t-1}} - 1$$

**Log returns (continuously compounded):**
$$r_t = \ln(P_t) - \ln(P_{t-1}) = \ln\left(\frac{P_t}{P_{t-1}}\right)$$

---

**Why Log Returns?**

**Advantages of log returns over simple returns:**

1. **Time-additivity**: Multi-period returns sum nicely
   $$r_{t,t+k} = \sum_{i=1}^{k} r_{t+i}$$
   
   For simple returns, you must compound: $(1+R_1)(1+R_2) - 1$

2. **Symmetry**: 
   - +50% followed by -50% (simple) gives: $1.5 \times 0.5 = 0.75$ (net loss!)
   - For small changes, log returns are symmetric: $\ln(1.5) \approx 0.405$, $\ln(0.5) \approx -0.693$

3. **Approximate percentage change**: For small $r$, $r_t \approx R_t$
   $$\ln(1 + R_t) \approx R_t \text{ when } |R_t| \text{ is small}$$

4. **Statistical properties**:
   - More likely to be normally distributed
   - Variance tends to be more stable over time
   - Better suited for econometric models (ARMA, GARCH)

5. **Cross-sectional properties**: If $r_A$ and $r_B$ are log returns, the portfolio return is approximately the weighted sum (for equal weights: $(r_A + r_B)/2$)

---

**Stata Implementation**

**The `ln()` function:**
- Computes natural logarithm (base $e$)
- Alternative: `log()` (same as `ln()`)
- For base 10: `log10()`

**Step-by-step process:**
```stata
gen log_sp500 = ln(sp500_close)    // Take log of price
gen sp500_return = D.log_sp500      // First difference of log
```

**What `D.` does:**
- `D.varname` = `varname - L.varname`
- For log_sp500: $\ln(P_t) - \ln(P_{t-1})$ = log return
- Automatically handles time series structure (requires `tsset` first)
- First observation becomes missing (no previous value)

**Alternative without `D.` operator:**
```stata
gen sp500_return = ln(sp500_close) - ln(L.sp500_close)
```
This is equivalent but less concise.

**Or in one step:**
```stata
gen sp500_return = D.ln(sp500_close)
```
Both approaches work; we use two steps for clarity.

---

**Why We Drop the Intermediate `log_sp500`:**

We created `log_sp500` only to compute returns. For most financial analysis:
- We care about **returns** (changes), not log levels
- Keeping unnecessary variables clutters the dataset
- If needed later, we can always recreate it from prices

**When to keep log levels:**
- Cointegration analysis (testing relationships between log price levels)
- Error correction models
- Long-run equilibrium analysis

---

**Checking the Results**

**The `summarize sp500_return, detail` command shows:**

**Expected properties of daily stock returns:**
- **Mean ≈ 0**: Stock returns have small average daily gain (maybe 0.03% to 0.05%)
- **Std Dev ≈ 1-2%**: Daily volatility typically 1-2% for broad indices
- **Skewness < 0**: Returns often left-skewed (large drops more common than large gains)
- **Kurtosis > 3**: "Fat tails" - extreme events more common than normal distribution
- **Min/Max**: Check for outliers (crashes: < -10%, rallies: > 10%)

**Common issues to check:**
- Returns > 100%: Likely data error (unless stock split not adjusted)
- Too many zeros: Possible stale prices or data issues  
- First observation missing: Normal (no previous price to compare)

In [ ]:
* Add variable labels
label var date "Date"
label var sp500_close "S&P 500 Closing Price"
label var sp500_return "S&P 500 Log Returns"

* Set time series
tsset date

* Check time series structure
tsset







Time variable: date, 04nov2015 to 03nov2025, but with gaps
        Delta: 1 day


Time variable: date, 04nov2015 to 03nov2025, but with gaps
        Delta: 1 day


### Step 6: Adding Variable Labels for Documentation

Proper documentation is crucial for reproducibility and collaboration. Variable labels serve as built-in documentation.

**The `label var` Command**

**Syntax:** `label variable varname "label text"`  
**Short form:** `label var varname "label text"`

**What it does:**
- Attaches a descriptive text label to a variable
- Appears in output from `describe`, `summarize`, graphs, tables
- Stored with the dataset when you `save`
- Maximum 80 characters

**Our commands:**
```stata
label var date "Date"
label var sp500_close "S&P 500 Closing Price"
label var sp500_return "S&P 500 Log Returns"
```

**Best practices:**
- **Be descriptive but concise**: "S&P 500 Closing Price" not just "Price"
- **Include units if relevant**: "GDP in billions of 2012 USD"
- **Specify transformations**: "Log Returns" not just "Returns"
- **Update labels if you transform**: If you create `log_gdp`, label it as such

**Benefits:**
- Future you will remember what variables mean
- Graphs automatically use labels for axes titles
- Regression output shows labels instead of cryptic variable names
- Collaborators can understand your data

---

**Checking Time Series Properties with `tsset`**

After setting up the time series structure earlier with `tsset date`, we can run `tsset` again without arguments to **check** the current settings:

**What `tsset` (no arguments) displays:**
```
Time variable: date, 01jan1990 to 31dec2023
        Delta: 1 day
```

**Information shown:**
- **Time variable**: Confirms which variable is the time index
- **Range**: First and last observation dates
- **Delta**: Time between consecutive observations (1 day for daily data)
- **Gaps**: Number of gaps (weekends/holidays for financial data)

**Why check this:**
- Confirms `tsset` worked correctly
- Verifies the time range (sample period)
- Identifies gaps (expected ~104 weekend days + ~10 holidays per year)

**Troubleshooting:**
- If you get "data not tsset": you need to run `tsset timevar`
- If you get "repeated time values": you have duplicates (use `duplicates drop`)
- If Delta is wrong: might have wrong time frequency or irregular data

In [ ]:
* Save processed S&P 500 data
save "data/processed/sp500_data.dta", replace

display "S&P 500 data processed and saved successfully!"
describe



file data/processed/sp500_data.dta saved

S&P 500 data processed and saved successfully!


Contains data from data/processed/sp500_data.dta
 Observations:         2,609                  
    Variables:             3                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
sp500_close     float   %9.0g                 S&P 500 Closing Price
date            float   %td                   Date
sp500_return    float   %9.0g                 S&P 500 Log Returns
--------------------------------------------------------------------------------
Sorted by: date


### Step 7: Saving the Processed Dataset

Now that we've cleaned and transformed the data, we need to save it for use in subsequent analyses.

**The `save` Command**

**Syntax:** `save filename [, options]`

**Key options:**
- `replace`: Overwrite file if it already exists (without this, Stata throws an error)
- `orphans`: Save even if value labels aren't attached to variables (rare use)
- `all`: Include all characteristics and stored results (advanced)

**Our command:**
```stata
save "data/processed/sp500_data.dta", replace
```

**What happens:**
- Saves current data in memory to a Stata `.dta` file
- Preserves:
  - All variables and observations
  - Variable labels
  - Variable formats
  - tsset settings (time series structure)
  - Sort order
  - Value labels (if any)
  - Data characteristics
- Location: `data/processed/` (separate from raw data)

---

**File Organization Best Practices**

**The raw vs. processed distinction:**

**Raw data** (`data/raw/`):
- Original, unmodified files from source (FRED, Bloomberg, etc.)
- **Never modify these files!**
- Keep them as a permanent record
- Can always regenerate processed data from raw

**Processed data** (`data/processed/`):
- Cleaned, transformed, ready-to-analyze versions
- Created by reproducible scripts/notebooks
- Can be deleted and regenerated
- Specific to your analysis needs

**Why this separation matters:**
- **Reproducibility**: Always know where data came from
- **Version control**: Can track changes to processing code
- **Collaboration**: Clear which files are source vs. derived
- **Data integrity**: Raw data never gets corrupted by analysis

---

**The `.dta` Format**

Stata's native binary format:
- **Efficient**: Compressed, fast to load
- **Comprehensive**: Stores metadata (labels, formats, etc.)
- **Portable**: Works across operating systems
- **Version-aware**: Stata handles different file versions

**Alternatives for sharing:**
- `.csv`: Universal but loses metadata (labels, formats)
- `.xlsx`: Good for viewing but inefficient for large datasets
- `.dta`: Best for Stata-to-Stata workflow

**Loading saved data:**
```stata
use "data/processed/sp500_data.dta", clear
```

---

**The `display` Command**

**Syntax:** `display [display_directive] [display_directive ...]`

**Simple use:**
```stata
display "S&P 500 data processed and saved successfully!"
```

**What it does:**
- Prints text to the Results window
- Useful for status messages in scripts
- Can also compute and display expressions

**More advanced uses:**
```stata
display "Number of observations: " _N
display "Mean return: " %6.4f r(mean)
display "Today's date: " %tdDDmonYY date("$S_DATE", "DMY")
```

**Display formats:**
- `%6.4f`: Fixed format (6 total width, 4 decimals)
- `%9.0g`: General format
- `%td`: Date format
- `_N`: System variable for number of observations

**Why use `display` in notebooks:**
- Provides feedback on workflow progress
- Confirms steps completed successfully
- Useful for debugging (print variable values)

---

**Final Verification**

The `describe` command at the end confirms:
- Number of observations and variables
- Variable names and types
- Storage format
- Variable labels we added
- Dataset is ready for analysis

**What to check:**
- ✓ Expected number of observations (roughly 252 trading days/year × years)
- ✓ Two variables: `date` and returns (we dropped log_sp500)
- ✓ Labels are present and correct
- ✓ File saved in correct location

<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">4) EUR/USD Exchange Rate Data</h2>

Process daily EUR/USD exchange rate data. This will be used for unit root testing.


## Processing Strategy for EUR/USD Data

The EUR/USD exchange rate data follows the same general workflow as the S&P 500 data we just processed:
1. Import CSV
2. Parse dates  
3. Check for duplicates and missing values
4. Transform the data
5. Add labels
6. Save

**Key Difference:** For exchange rates, we'll create **multiple transformations** and keep them all:
- **Original level** (`eurusd`): For level analysis, cointegration tests
- **Log level** (`lneurusd`): For unit root tests in logs
- **First difference** (`d_lneurusd`): For testing stationarity, returns analysis

This is because unit root testing (which we'll do in the next notebook) often requires examining the series in multiple forms.

**Why EUR/USD?**
- Major global exchange rate (most liquid currency pair)
- Benchmark for foreign exchange analysis
- Classic example for unit root tests and random walk hypothesis
- Relevant for international finance and macroeconomics

In [ ]:
* Load EUR/USD raw data
import delimited "data/raw/DEXUSEU.csv", clear

* Display first few observations
list in 1/10

* Check data structure
describe



(encoding automatically selected: ISO-8859-1)
(2 vars, 7,000 obs)


     +----------------------+
     | observat~e   dexuseu |
     |----------------------|
  1. | 1999-01-04    1.1812 |
  2. | 1999-01-05     1.176 |
  3. | 1999-01-06    1.1636 |
  4. | 1999-01-07    1.1672 |
  5. | 1999-01-08    1.1554 |
     |----------------------|
  6. | 1999-01-11    1.1534 |
  7. | 1999-01-12    1.1548 |
  8. | 1999-01-13    1.1698 |
  9. | 1999-01-14    1.1689 |
 10. | 1999-01-15    1.1591 |
     +----------------------+


Contains data
 Observations:         7,000                  
    Variables:             2                  
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
observation_d~e str10   %10s                  
dexuseu         float   %9.0g                 DEXUSEU
------

In [ ]:
* Parse dates from observation_date column
gen date_numeric = date(observation_date, "YMD")
drop observation_date

* Format as Stata daily date
format date_numeric %td
rename date_numeric date

* Check for duplicates
duplicates report date
duplicates drop date, force

* Sort by date
sort date








Duplicates in terms of date

--------------------------------------
   Copies | Observations       Surplus
----------+---------------------------
        1 |         7000             0
--------------------------------------


Duplicates in terms of date

(0 observations are duplicates)



In [ ]:
* Rename exchange rate variable (column name matches filename: DEXUSEU)
rename dexuseu eurusd

* Check for missing values
count if missing(eurusd)
summarize eurusd, detail




  270


                           DEXUSEU
-------------------------------------------------------------
      Percentiles      Smallest
 1%        .8598           .827
 5%        .9037          .8271
10%       .98485          .8364       Obs               6,730
25%       1.0844          .8364       Sum of wgt.       6,730

50%       1.1702                      Mean           1.182768
                        Largest       Std. dev.      .1540254
75%       1.2994         1.5923
90%      1.37945         1.5924       Variance       .0237238
95%       1.4399         1.5978       Skewness       .1009166
99%       1.5569          1.601       Kurtosis       2.676944


### Understanding Exchange Rate Data

**What is DEXUSEU?**
- FRED ticker for U.S. Dollars to One Euro exchange rate
- **Interpretation**: How many USD needed to buy 1 EUR
  - If DEXUSEU = 1.10, then 1 EUR = 1.10 USD
  - Increase means EUR appreciation (USD depreciation)
  - Decrease means EUR depreciation (USD appreciation)

**Missing Values in Exchange Rate Data:**

Exchange rate data has missing values for the same reason as stock prices:
- Weekends: No trading Saturday/Sunday
- Holidays: Both U.S. and European holidays
- Market closures: Occasional disruptions

The `count if missing()` command helps us quantify this. We expect:
- ~104 weekend days per year
- ~15-20 holidays per year (varies by year and whether holidays fall on weekdays)
- Total: ~30-35% of calendar days missing (roughly 130 missing days per year)

This is **normal and expected** for financial data. Time series operators in Stata handle gaps correctly after `tsset`.

In [ ]:
* Set time series (required before using D. operator)
tsset date

* Create log exchange rate and first difference
gen lneurusd = ln(eurusd)
gen d_lneurusd = D.lneurusd

* Check summary statistics
summarize eurusd lneurusd d_lneurusd, detail




Time variable: date, 04jan1999 to 31oct2025, but with gaps
        Delta: 1 day

(270 missing values generated)

(1,742 missing values generated)


                           DEXUSEU
-------------------------------------------------------------
      Percentiles      Smallest
 1%        .8598           .827
 5%        .9037          .8271
10%       .98485          .8364       Obs               6,730
25%       1.0844          .8364       Sum of wgt.       6,730

50%       1.1702                      Mean           1.182768
                        Largest       Std. dev.      .1540254
75%       1.2994         1.5923
90%      1.37945         1.5924       Variance       .0237238
95%       1.4399         1.5978       Skewness       .1009166
99%       1.5569          1.601       Kurtosis       2.676944

                          lneurusd
-------------------------------------------------------------
      Percentiles      Smallest
 1%    -.1510555      -.1899506
 5%    -.1012578      -.1898

### Exchange Rate Transformations: Theory and Application

For exchange rates, we create two transformations that are essential for time series econometrics.

---

**Transformation 1: Log Exchange Rate**

**Theory: Why take logs of exchange rates?**

The log transformation has several important properties for exchange rates:

1. **Symmetry property:**
   - If EUR/USD goes from 1.00 to 1.10 (+10%), the reverse rate USD/EUR goes from 1.00 to 0.909 (-9.1%)
   - In logs: $\ln(1.10) = 0.095$ and $\ln(0.909) = -0.095$ (symmetric!)
   - This makes appreciation and depreciation symmetric

2. **Interpretability:**
   - Small changes in log exchange rate ≈ percentage changes
   - $\Delta \ln(S_t) \approx \frac{S_t - S_{t-1}}{S_{t-1}}$ for small changes

3. **Statistical properties:**
   - Log exchange rates often more normally distributed than levels
   - Variance stabilization (reduces heteroskedasticity)

4. **Theoretical models:**
   - Many exchange rate theories (PPP, UIP, monetary models) expressed in logs
   - Facilitates elasticity interpretation in regressions

**Stata implementation:**
```stata
gen lneurusd = ln(eurusd)
```

The `ln()` function computes the natural logarithm (base $e$).

---

**Transformation 2: First Difference of Log Exchange Rate**

**Theory: Differencing for stationarity**

**Definition:** The first difference operator:
$$\Delta x_t = x_t - x_{t-1}$$

Applied to log exchange rate:
$$\Delta \ln(S_t) = \ln(S_t) - \ln(S_{t-1}) = \ln\left(\frac{S_t}{S_{t-1}}\right)$$

This is the **continuously compounded return** or **growth rate** of the exchange rate.

**Why difference?**

**Stationarity concept:**
- A time series is **stationary** if its statistical properties (mean, variance, autocorrelations) don't change over time
- **Non-stationary** series often have trends, changing variance, or unit roots
- Most econometric methods require stationary data

**Integration and unit roots:**
- A series with a **unit root** is integrated of order 1, denoted $I(1)$
- If $x_t \sim I(1)$, then $\Delta x_t \sim I(0)$ (stationary)
- Exchange rates are typically $I(1)$: the level has a unit root, but the change is stationary

**Random walk hypothesis:**
$$S_t = S_{t-1} + \varepsilon_t$$

where $\varepsilon_t$ is white noise. This implies:
$$\Delta S_t = \varepsilon_t \sim I(0)$$

For log exchange rate:
$$\ln(S_t) = \ln(S_{t-1}) + u_t$$
$$\Delta \ln(S_t) = u_t$$

The random walk hypothesis suggests exchange rate changes are unpredictable (efficient markets).

**Stata implementation:**
```stata
gen d_lneurusd = D.lneurusd
```

The `D.` operator:
- Requires `tsset` first
- Computes `varname - L.varname`
- Automatically makes first observation missing (no previous value)
- Handles gaps correctly (weekends, holidays)

**Alternative (equivalent):**
```stata
gen d_lneurusd = lneurusd - L.lneurusd
```

---

**Why Keep Both `lneurusd` and `d_lneurusd`?**

Different tests and models require different transformations:

1. **Unit root tests** (Dickey-Fuller, Phillips-Perron):
   - Test whether level has unit root
   - Test whether first difference is stationary
   - Need both to determine order of integration

2. **Cointegration analysis:**
   - Tests long-run relationships between levels
   - Requires non-stationary levels

3. **ARMA/VAR models:**
   - Typically use stationary (differenced) data
   - But need levels to interpret long-run effects

4. **Forecasting:**
   - Forecast differences, then cumulate to get level forecasts

---

**Interpreting the Summary Statistics**

When we run `summarize eurusd lneurusd d_lneurusd, detail`, look for:

**For `eurusd` (level):**
- Range over sample period (e.g., 0.85 to 1.60 from 2000-2023)
- High persistence (if we compute autocorrelations)

**For `lneurusd` (log level):**
- Similar but in log scale
- Mean ≈ 0 to 0.4 (since ln(1.0) = 0, ln(1.5) ≈ 0.4)

**For `d_lneurusd` (first difference / returns):**
- **Mean ≈ 0**: Exchange rate has no persistent drift (on average)
- **Std dev ≈ 0.5-1%**: Daily volatility of exchange rate
- **Skewness**: Should be close to 0 (symmetric shocks)
- **Kurtosis > 3**: Fat tails (occasional large jumps)
- **Min/Max**: Look for extreme events (financial crises, interventions)

In [ ]:
* Add variable labels
label var date "Date"
label var eurusd "EUR/USD Exchange Rate"
label var lneurusd "Log EUR/USD Exchange Rate"
label var d_lneurusd "First Difference of Log EUR/USD"

* Set time series
tsset date

* Check time series structure
tsset








Time variable: date, 04jan1999 to 31oct2025, but with gaps
        Delta: 1 day


Time variable: date, 04jan1999 to 31oct2025, but with gaps
        Delta: 1 day


In [ ]:
* Save processed EUR/USD data
save "data/processed/eurusd_data.dta", replace

display "EUR/USD data processed and saved successfully!"
describe



file data/processed/eurusd_data.dta saved

EUR/USD data processed and saved successfully!


Contains data from data/processed/eurusd_data.dta
 Observations:         7,000                  
    Variables:             4                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
eurusd          float   %9.0g                 EUR/USD Exchange Rate
date            float   %td                   Date
lneurusd        float   %9.0g                 Log EUR/USD Exchange Rate
d_lneurusd      float   %9.0g                 First Difference of Log EUR/USD
--------------------------------------------------------------------------------
Sorted by: date


<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">5) Macro-Finance Data</h2>

Process multiple macroeconomic and financial series and merge them into a single dataset for VAR analysis. We'll process each series separately, then merge on common dates.

### 5.1 Real GDP (Quarterly)


## The Multi-Frequency Challenge

Processing macroeconomic and financial data presents a unique challenge: **different variables are available at different frequencies**.

**Our data frequencies:**
- **Quarterly**: Real GDP (GDPC1)
- **Monthly**: CPI, Interest Rate, Unemployment
- **Daily**: Stock prices (S&P 500)

**The fundamental problem:**
- VAR (Vector Autoregression) models require all variables at the **same frequency**
- We cannot directly combine quarterly and monthly data
- We must choose a common frequency and convert all series

**Our strategy:**
1. **Convert daily → monthly**: Aggregate S&P 500 to monthly returns
2. **Convert quarterly → monthly**: Interpolate or repeat GDP values
3. **Keep monthly as is**: CPI, interest rates, unemployment
4. **Merge all series**: Create unified monthly dataset

**Alternative approaches:**

1. **Mixed-frequency models** (MIDAS, MF-VAR):
   - Advanced techniques that handle multiple frequencies
   - More complex to implement and interpret
   - Beyond scope of this course

2. **Aggregate everything to quarterly:**
   - Simpler, matches GDP frequency
   - Loses information from higher-frequency data
   - Only 4 observations per year (small sample)

3. **Use only same-frequency variables:**
   - Avoids conversion issues
   - Limits scope of analysis (can't include GDP with monthly vars)

**We choose monthly frequency because:**
- Good balance: enough observations for VAR estimation (12/year)
- Most macro variables available monthly
- Standard practice in macro-financial VAR analysis
- Facilitates policy analysis (monetary policy typically monthly)

In [ ]:
* Load Real GDP data (quarterly)
import delimited "data/raw/GDPC1.csv", clear

* Parse dates from observation_date column
gen date_numeric = date(observation_date, "YMD")
drop observation_date

* Convert to quarterly format
* First convert to daily, then to quarterly
format date_numeric %td
gen year = year(date_numeric)
gen month = month(date_numeric)
gen quarter = ceil(month/3)
gen date_q = yq(year, quarter)
format date_q %tq

* Rename GDP variable (column name matches filename: GDPC1)
rename gdpc1 gdp

* Set time series (required before using D. operator)
rename date_q date
tsset date

* Create log GDP and growth rate
gen lngdp = ln(gdp)
gen gdp_growth = D.lngdp

* Keep only quarterly date and variables
keep date gdp lngdp gdp_growth

* Remove duplicates and sort
duplicates drop date, force
sort date

* Save temporary GDP file
save "data/processed/temp_gdp.dta", replace

describe
summarize gdp lngdp gdp_growth, detail



(encoding automatically selected: ISO-8859-1)
(2 vars, 314 obs)












Time variable: date, 1947q1 to 2025q2
        Delta: 1 quarter


(1 missing value generated)



Duplicates in terms of date

(0 observations are duplicates)


(file data/processed/temp_gdp.dta not found)
file data/processed/temp_gdp.dta saved


Contains data from data/processed/temp_gdp.dta
 Observations:           314                  
    Variables:             4                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
gdp             float   %9.0g                 GDPC1
date            float   %tq                   
lngdp           float   %9.0g                 
gdp_growth      float   %9.0g                 
----------------------------------------------------------------

### Processing Real GDP: Quarterly to Monthly Conversion

**About the GDP Data (GDPC1):**
- **Definition**: Real Gross Domestic Product
- **Units**: Billions of Chained 2017 Dollars
- **Frequency**: Quarterly (Q1, Q2, Q3, Q4)
- **Seasonal adjustment**: Seasonally Adjusted Annual Rate (SAAR)
- **Source**: U.S. Bureau of Economic Analysis via FRED

**Why "Real" GDP?**
- Adjusted for inflation (using 2017 as base year)
- Measures actual output, not just price changes
- Allows comparison across time periods
- "Chained" means uses chain-weighted price index (more accurate than fixed weights)

---

**Quarterly Date Handling in Stata**

**The `%tq` format:**
- Quarterly dates stored as integers (quarters since 1960q1)
- 1960q1 = 0, 1960q2 = 1, 1960q3 = 2, etc.
- Display format: `%tq` shows as "2020q1", "2020q2", etc.

**Date extraction functions:**
```stata
gen year = year(date_numeric)      // Extract year from daily date
gen month = month(date_numeric)    // Extract month (1-12)
gen quarter = ceil(month/3)        // Compute quarter (1-4)
```

**The `ceil()` function** (ceiling):
- Rounds UP to nearest integer
- January (month=1): ceil(1/3) = ceil(0.33) = 1 → Q1
- February (month=2): ceil(2/3) = ceil(0.67) = 1 → Q1
- March (month=3): ceil(3/3) = ceil(1.00) = 1 → Q1
- April (month=4): ceil(4/3) = ceil(1.33) = 2 → Q2
- etc.

**Alternative:** `quarter()` function:
```stata
gen quarter = quarter(date_numeric)
```
This is simpler and equivalent!

**The `yq()` function:**
- Creates quarterly date from year and quarter
- `yq(2020, 1)` creates 2020q1
- Returns integer that Stata interprets as quarterly date when formatted with `%tq`

---

**GDP Growth Rate Calculation**

**Theory: Why use growth rates?**

**Level vs. Growth:**
- **GDP level** ($Y_t$): Actual output (e.g., $23,000 billion)
  - Non-stationary: has trend (economy grows over time)
  - Scale-dependent: hard to compare across countries or time periods
  
- **GDP growth rate** ($g_t$): Percentage change
  - Closer to stationary (though can have business cycle persistence)
  - Scale-free: comparable across contexts
  - What matters for economic analysis (expansions vs. recessions)

**Computation:**

**Discrete growth rate:**
$$g_t = \frac{Y_t - Y_{t-1}}{Y_{t-1}} = \frac{Y_t}{Y_{t-1}} - 1$$

**Log growth rate (our choice):**
$$g_t = \ln(Y_t) - \ln(Y_{t-1}) = \Delta \ln(Y_t)$$

**For small changes:** $\Delta \ln(Y_t) \approx \frac{Y_t - Y_{t-1}}{Y_{t-1}}$

**Advantages of log growth:**
- Time-additive: 4-quarter growth = sum of 4 quarterly growth rates
- Symmetric (as discussed for returns)
- More likely to be normally distributed

**Annualized growth:**
- Our calculation gives **quarterly growth** (e.g., 0.005 = 0.5% per quarter)
- To annualize: multiply by 4 (e.g., 0.5% × 4 = 2% annual rate)
- BEA reports "annualized" growth (compounded: $(1+g_q)^4 - 1$)
- We keep quarterly for consistency with data frequency

**Stata implementation:**
```stata
gen lngdp = ln(gdp)
gen gdp_growth = D.lngdp
```

**Why `tsset` before creating growth:**
- `D.` operator requires time series structure
- Ensures correct time alignment (t vs. t-1)
- Handles gaps properly (though quarterly GDP rarely has gaps)

---

**Temporary File Strategy**

**Why save temporary files?**
- Each series processed separately (clearer, easier to debug)
- Then merged together at the end
- Allows checking each series individually
- Can rerun parts without reprocessing everything

**The `save` command with temporary files:**
```stata
save "data/processed/temp_gdp.dta", replace
```

**File naming convention:**
- `temp_`: Indicates file is intermediate, not final output
- Will be deleted after merging
- Clearly distinguishes from final processed data

**Best practice:**
- Process each series completely (import → clean → transform → save)
- Verify each series individually
- Merge all at the end
- Clean up temporary files

### 5.2 CPI (Monthly)


In [ ]:
* Load CPI data (monthly)
import delimited "data/raw/CPIAUCSL.csv", clear

* Parse dates from observation_date column
gen date_numeric = date(observation_date, "YMD")
drop observation_date

* Convert to monthly format
format date_numeric %td
gen year = year(date_numeric)
gen month = month(date_numeric)
gen date_m = ym(year, month)
format date_m %tm

* Rename CPI variable (column name matches filename: CPIAUCSL)
rename cpiaucsl cpi

* Set time series (required before using D. operator)
rename date_m date
tsset date

* Create inflation rate (monthly change, annualized percentage)
* First create log of CPI, then take first difference
gen ln_cpi = ln(cpi)
gen inflation = 100 * D.ln_cpi

* Keep only monthly date and variables
keep date cpi inflation

* Remove duplicates and sort
duplicates drop date, force
sort date

* Save temporary CPI file
save "data/processed/temp_cpi.dta", replace

describe
summarize cpi inflation, detail



(encoding automatically selected: ISO-8859-1)
(2 vars, 945 obs)











Time variable: date, 1947m1 to 2025m9
        Delta: 1 month


(1 missing value generated)



Duplicates in terms of date

(0 observations are duplicates)


(file data/processed/temp_cpi.dta not found)
file data/processed/temp_cpi.dta saved


Contains data from data/processed/temp_cpi.dta
 Observations:           945                  
    Variables:             3                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
cpi             float   %9.0g                 CPIAUCSL
date            float   %tm                   
inflation       float   %9.0g                 
--------------------------------------------------------------------------------
Sorted by: date


            

### Processing CPI and Computing Inflation

**About the CPI Data (CPIAUCSL):**
- **Definition**: Consumer Price Index for All Urban Consumers: All Items
- **Coverage**: U.S. City Average
- **Base period**: Index where 1982-84 = 100
- **Frequency**: Monthly
- **Seasonal adjustment**: Seasonally Adjusted
- **Source**: U.S. Bureau of Labor Statistics via FRED

**What CPI measures:**
- Average change in prices paid by urban consumers for a market basket of goods and services
- Includes: food, clothing, shelter, fuel, transportation, medical care, etc.
- Covers about 93% of U.S. population
- Most widely used measure of inflation

---

**Monthly Date Format**

**The `%tm` format:**
- Monthly dates stored as integers (months since 1960m1)
- 1960m1 = 0, 1960m2 = 1, etc.
- Display format: `%tm` shows as "2020m1", "2020m2", etc.

**The `ym()` function:**
```stata
gen date_m = ym(year, month)
```
- Combines year and month into Stata monthly date
- `ym(2020, 3)` creates 2020m3 (March 2020)
- Returns integer interpretable as monthly date with `%tm` format

**Why extract year and month first:**
- FRED gives daily date (even though CPI is monthly)
- E.g., "2020-03-01" for March 2020 CPI
- We extract components to create proper monthly date
- Ensures all March values map to same 2020m3

---

**Inflation Calculation: Theory and Practice**

**What is inflation?**

Inflation ($\pi_t$) is the rate of change in the price level.

**Discrete inflation rate:**
$$\pi_t = \frac{CPI_t - CPI_{t-1}}{CPI_{t-1}} \times 100\%$$

**Log approximation (our method):**
$$\pi_t = 100 \times (\ln(CPI_t) - \ln(CPI_{t-1})) = 100 \times \Delta \ln(CPI_t)$$

**Why these are approximately equal:**

For small changes, using Taylor expansion:
$$\ln(1 + x) \approx x \text{ when } |x| \text{ is small}$$

Therefore:
$$\ln(CPI_t) - \ln(CPI_{t-1}) = \ln\left(\frac{CPI_t}{CPI_{t-1}}\right) = \ln\left(1 + \frac{CPI_t - CPI_{t-1}}{CPI_{t-1}}\right) \approx \frac{CPI_t - CPI_{t-1}}{CPI_{t-1}}$$

**Example:**
- If CPI goes from 250 to 252.5 (1% increase):
- Discrete: $(252.5 - 250)/250 = 0.01 = 1\%$
- Log: $\ln(252.5) - \ln(250) = 0.00995 \approx 1\%$

**Why multiply by 100:**
- Converts to percentage points
- 0.01 × 100 = 1 (meaning 1%)
- Standard presentation format
- Easier interpretation

**Monthly vs. Annualized inflation:**
- Our calculation: **monthly inflation rate** (percentage change from previous month)
- To annualize (roughly): multiply by 12
- More precisely: $(1 + \pi_{monthly})^{12} - 1$
- Federal Reserve targets 2% **annual** inflation (≈ 0.167% monthly)

**Stata implementation:**
```stata
gen ln_cpi = ln(cpi)
gen inflation = 100 * D.ln_cpi
```

**Step by step:**
1. `ln(cpi)`: Take natural log of CPI index
2. `D.ln_cpi`: Compute first difference (requires `tsset` first)
3. `100 *`: Convert to percentage points

**Why take log first, then difference:**
- `D.ln(cpi)` = $\ln(CPI_t) - \ln(CPI_{t-1})$ = log change
- This is NOT the same as `ln(D.cpi)` = $\ln(CPI_t - CPI_{t-1})$ (not useful!)
- Order matters: log-difference ≠ difference-log

---

**Interpreting CPI and Inflation**

**Expected patterns:**

**For CPI (level):**
- **Always increasing** (except rare deflation periods)
- Value around 250-300 in recent years (base 1982-84 = 100)
- Non-stationary: has a trend

**For inflation (first difference):**
- **Mean**: Around 0.15-0.25% monthly (2-3% annualized)
- **Range**: Typically -0.5% to +1% monthly
  - Negative: deflation (rare in U.S. since 1950s)
  - > 1% monthly: high inflation period (1970s, early 1980s, 2021-2022)
- **Volatility**: Varies over time
  - "Great Moderation" (1990s-2007): low, stable inflation
  - Recent years: more volatile
- **Persistence**: Inflation is autocorrelated (past inflation predicts future)

**Uses in macroeconomic analysis:**
- **Monetary policy**: Fed targets inflation around 2% annual
- **Real vs. nominal**: Deflate nominal variables using CPI
- **VAR analysis**: Inflation as endogenous variable with interest rates, GDP
- **Phillips curve**: Relationship between inflation and unemployment

### 5.3 Interest Rate (Monthly)


In [ ]:
* Load interest rate data (Fed Funds Rate - monthly)
import delimited "data/raw/FEDFUNDS.csv", clear

* Parse dates from observation_date column
gen date_numeric = date(observation_date, "YMD")
drop observation_date

* Convert to monthly format
format date_numeric %td
gen year = year(date_numeric)
gen month = month(date_numeric)
gen date_m = ym(year, month)
format date_m %tm

* Rename interest rate variable (column name matches filename: FEDFUNDS)
rename fedfunds interest_rate

* Keep only monthly date and variables
keep date_m interest_rate

* Rename date variable
rename date_m date

* Remove duplicates and sort
duplicates drop date, force
sort date

* Save temporary interest rate file
save "data/processed/temp_interest.dta", replace

describe
summarize interest_rate, detail



(encoding automatically selected: ISO-8859-1)
(2 vars, 856 obs)












Duplicates in terms of date

(0 observations are duplicates)


(file data/processed/temp_interest.dta not found)
file data/processed/temp_interest.dta saved


Contains data from data/processed/temp_interest.dta
 Observations:           856                  
    Variables:             2                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
interest_rate   float   %9.0g                 FEDFUNDS
date            float   %tm                   
--------------------------------------------------------------------------------
Sorted by: date


                          FEDFUNDS
-------------------------------------------------------------
      Percentiles      Smallest
 1%     

### Processing Interest Rates

**About FEDFUNDS:**
- **Definition**: Effective Federal Funds Rate
- **Units**: Percent per annum
- **Frequency**: Monthly (average of daily values)
- **Source**: Federal Reserve Board via FRED

**What is the Federal Funds Rate?**

The federal funds rate is the **interest rate** at which depository institutions (banks and credit unions) lend reserve balances to other depository institutions overnight.

**Key characteristics:**
- **Overnight rate**: Very short-term (1-day) lending
- **Interbank market**: Banks lending to each other
- **Unsecured**: No collateral required
- **Market-determined**: Result of supply and demand (but heavily influenced by Fed)

**Why it matters:**
- **Monetary policy instrument**: Fed's primary tool for controlling monetary conditions
- **Target rate**: FOMC sets a target range, market rate gravitates toward it
- **Benchmark**: Influences all other interest rates in the economy
  - Prime rate = Fed Funds + ~3%
  - Mortgage rates, corporate bonds, etc. move with Fed Funds
- **Economic indicator**: Low rates → stimulus, High rates → restraint

**Historical context:**
- Near 20% in early 1980s (Volcker fighting inflation)
- Near 0% in 2008-2015 (Great Recession response)
- Near 0% in 2020-2021 (COVID-19 response)
- Rapid increases in 2022-2023 (fighting inflation)

**No transformation needed:**
- Already in interpretable units (%)
- Already stationary (tends to mean-revert, though slowly)
- Use in levels for VAR analysis

**For time series analysis:**
- Interest rates typically $I(1)$ or near unit root (high persistence)
- First difference would be change in interest rate (useful for some models)
- We keep the level for standard macro VAR

### 5.4 Unemployment Rate (Monthly)


In [ ]:
* Load unemployment rate data (monthly)
import delimited "data/raw/UNRATE.csv", clear

* Parse dates from observation_date column
gen date_numeric = date(observation_date, "YMD")
drop observation_date

* Convert to monthly format
format date_numeric %td
gen year = year(date_numeric)
gen month = month(date_numeric)
gen date_m = ym(year, month)
format date_m %tm

* Rename unemployment variable (column name matches filename: UNRATE)
rename unrate unemployment

* Keep only monthly date and variables
keep date_m unemployment

* Rename date variable
rename date_m date

* Remove duplicates and sort
duplicates drop date, force
sort date

* Save temporary unemployment file
save "data/processed/temp_unemployment.dta", replace

describe
summarize unemployment, detail



(encoding automatically selected: ISO-8859-1)
(2 vars, 932 obs)












Duplicates in terms of date

(0 observations are duplicates)


(file data/processed/temp_unemployment.dta not found)
file data/processed/temp_unemployment.dta saved


Contains data from data/processed/temp_unemployment.dta
 Observations:           932                  
    Variables:             2                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
unemployment    float   %9.0g                 UNRATE
date            float   %tm                   
--------------------------------------------------------------------------------
Sorted by: date


                           UNRATE
-------------------------------------------------------------
      Percentiles      Smallest

### Processing Unemployment Rate

**About UNRATE:**
- **Definition**: Civilian Unemployment Rate
- **Units**: Percent of labor force
- **Frequency**: Monthly
- **Seasonal adjustment**: Seasonally Adjusted
- **Source**: U.S. Bureau of Labor Statistics via FRED

**What is the unemployment rate?**

$$\text{Unemployment Rate} = \frac{\text{Number of Unemployed}}{\text{Labor Force}} \times 100\%$$

Where:
- **Unemployed**: People without a job, actively looking for work, and available to work
- **Labor Force**: Employed + Unemployed (excludes those not looking for work)

**Key characteristics:**
- **Countercyclical**: Rises in recessions, falls in expansions
- **Lagging indicator**: Often continues rising after recession ends
- **Natural rate**: Long-run average around 4-5% in U.S. (varies by period)
- **Structural component**: Some unemployment always exists (job search, mismatch)

**Historical patterns:**
- Low: ~3.5% (late 1960s, 2019)
- High: ~10% (Great Recession 2009, early 1980s)
- COVID shock: Spiked to ~14.7% in April 2020

**Use in macroeconomic analysis:**

1. **Phillips Curve**: Relationship between unemployment and inflation
   - Original: $\pi_t = f(u_t)$ - inverse relationship
   - Expectations-augmented: $\pi_t = \pi_t^e - \alpha(u_t - u_t^*)$
   - Where $u_t^*$ is natural rate (NAIRU)

2. **Okun's Law**: Relationship between GDP growth and unemployment changes
   - Approximately: $\Delta u_t \approx -0.5 \times (g_t - 3\%)$
   - GDP growth above trend → unemployment falls

3. **Monetary policy**: Fed's dual mandate (price stability + maximum employment)
   - Fed responds to unemployment in setting interest rates
   - VAR with unemployment, inflation, interest rates captures policy response

**No transformation needed:**
- Already in percentage points (interpretable units)
- Relatively stationary around time-varying natural rate
- Use in levels for VAR (though some researchers difference it)

**Expected statistics:**
- **Mean**: Depends on sample period (4-6% typical)
- **Range**: 3% to 15% (covering normal times and severe recessions)
- **Persistence**: Highly autocorrelated (changes slowly)
- **Seasonal adjustment**: Important (unemployment varies by season)

### 5.5 Stock Index Returns (Monthly from S&P 500)

Aggregate daily S&P 500 data to monthly returns.


In [ ]:
* Load processed S&P 500 data
use "data/processed/sp500_data.dta", clear

* Convert daily date to monthly
gen year = year(date)
gen month = month(date)
gen date_m = ym(year, month)
format date_m %tm

* Aggregate to monthly: take last observation of each month
* For returns, we can sum daily returns to get monthly returns
bysort date_m: egen stock_return = total(sp500_return)

* Keep only one observation per month (last day of month)
bysort date_m: keep if _n == _N

* Keep only monthly date and return
keep date_m stock_return

* Rename date variable
rename date_m date

* Sort
sort date

* Save temporary stock return file
save "data/processed/temp_stock.dta", replace

describe
summarize stock_return, detail









(2,488 observations deleted)




(file data/processed/temp_stock.dta not found)
file data/processed/temp_stock.dta saved


Contains data from data/processed/temp_stock.dta
 Observations:           121                  
    Variables:             2                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
date            float   %tm                   
stock_return    float   %9.0g                 
--------------------------------------------------------------------------------
Sorted by: date


                        stock_return
-------------------------------------------------------------
      Percentiles      Smallest
 1%     -.100915      -.1157656
 5%    -.0583563       -.100915
10%     -.043582       -.088582       Obs                 1

### Aggregating Daily Returns to Monthly Frequency

Now we need to convert the S&P 500 daily data to monthly frequency to match our macroeconomic variables.

**The challenge:**
- We have daily stock returns (about 252 trading days/year)
- We need monthly returns (12 observations/year)
- How do we aggregate?

---

**Aggregation Methods for Returns**

**Option 1: End-of-month prices (Discrete return)**
- Take last price of month $t$ and last price of month $t-1$
- Compute: $R_{monthly} = \frac{P_{end,t} - P_{end,t-1}}{P_{end,t-1}}$
- **Pros**: Simple, uses actual observed prices
- **Cons**: Ignores intra-month information

**Option 2: Compound daily returns (Discrete)**
- Product of daily gross returns: $(1+r_1)(1+r_2)...(1+r_n) - 1$
- **Pros**: Accurate if returns are simple returns
- **Cons**: Complicated, compounding errors

**Option 3: Sum log returns (Our method)**
- Sum daily log returns: $r_{monthly} = \sum_{i=1}^{n} r_{daily,i}$
- **Pros**: 
  - Mathematically correct for log returns (time-additive property!)
  - Uses all available information
  - Simple implementation
- **Cons**: None for log returns

---

**Why Summing Log Returns Works**

**Mathematical property of log returns:**

$$r_t = \ln(P_t) - \ln(P_{t-1})$$

For multiple periods:
$$r_{1,n} = \ln(P_n) - \ln(P_1) = \sum_{i=1}^{n} (\ln(P_i) - \ln(P_{i-1})) = \sum_{i=1}^{n} r_i$$

**Example:** March 2020 with 21 trading days
- Day 1: $r_1 = 0.02$ (2% gain)
- Day 2: $r_2 = -0.03$ (-3% loss)
- ...
- Day 21: $r_{21} = 0.01$ (1% gain)

**Monthly return:**
$$r_{March} = r_1 + r_2 + ... + r_{21}$$

This equals $\ln(P_{March31}) - \ln(P_{Feb28})$, exactly what we want!

---

**Stata Implementation**

**Step 1: Create monthly date from daily date**
```stata
gen year = year(date)
gen month = month(date)
gen date_m = ym(year, month)
format date_m %tm
```

**Step 2: Aggregate returns within each month**
```stata
bysort date_m: egen stock_return = total(sp500_return)
```

**The `bysort` prefix:**
- `bysort varlist:` performs command separately for each value of varlist
- Here: operates on each unique value of `date_m` (each month)
- Equivalent to looping over months, but more efficient

**The `egen` command with `total()` function:**
- `egen`: Extended generate - creates variables using functions
- `total()`: Sums values within each group
- Different from `sum()` (running sum)

**What happens:**
- For each month (e.g., 2020m3):
  - Finds all daily observations in that month (21 trading days)
  - Sums their returns: $r_1 + r_2 + ... + r_{21}$
  - Assigns this sum to all observations in that month
  - Result: all 21 days have same `stock_return` value

**Step 3: Keep one observation per month**
```stata
bysort date_m: keep if _n == _N
```

**Understanding `_n` and `_N`:**
- `_n`: Current observation number within group (1, 2, 3, ...)
- `_N`: Total number of observations in group

**What `_n == _N` means:**
- Keep only if current obs number equals total obs in group
- I.e., keep the **last observation** in each month
- Result: One observation per month (the last trading day)

**Alternative (equivalent):**
```stata
bysort date_m: keep if _n == 1  // Keep first day of month
```
Doesn't matter which day we keep since they all have the same aggregated return value!

---

**Expected Results**

After aggregation:
- ~12 observations per year instead of ~252
- Monthly returns are larger magnitude than daily (more time to accumulate)
- Typical monthly return: -5% to +10% (much wider range than daily)
- Monthly volatility: ~3-5% standard deviation
- Still approximately normally distributed (but with fat tails)

### 5.6 Merge All Macro-Finance Series

Merge all monthly series together. Note: GDP is quarterly, so we'll merge it separately or convert to monthly.


In [ ]:
* First, process GDP quarterly to monthly conversion
* Load GDP quarterly data
use "data/processed/temp_gdp.dta", clear

* Extract year and quarter from quarterly date
gen year = year(date)
gen quarter = quarter(date)

* Create monthly dates for all three months in each quarter
expand 3
bysort date: gen month_in_quarter = _n  // 1, 2, or 3
gen month = (quarter - 1) * 3 + month_in_quarter
gen date_m = ym(year, month)
format date_m %tm

* Keep only one observation per month (carry forward quarterly values)
bysort date_m: keep if _n == 1

* Drop old quarterly date variable before renaming
drop date

* Rename and keep variables
rename date_m date
keep date gdp lngdp gdp_growth

* Sort by date
sort date

* Save temporary expanded GDP
save "data/processed/temp_gdp_monthly.dta", replace

* Now start with CPI as base (monthly) and merge all series
use "data/processed/temp_cpi.dta", clear

* Merge with interest rate
merge 1:1 date using "data/processed/temp_interest.dta", nogen

* Merge with unemployment
merge 1:1 date using "data/processed/temp_unemployment.dta", nogen

* Merge with stock returns
merge 1:1 date using "data/processed/temp_stock.dta", nogen

* Merge with GDP (monthly version)
merge 1:1 date using "data/processed/temp_gdp_monthly.dta", nogen

* Sort by date
sort date






(628 observations created)





(930 observations deleted)





(file data/processed/temp_gdp_monthly.dta not found)
file data/processed/temp_gdp_monthly.dta saved



    Result                      Number of obs
    -----------------------------------------
    Not matched                            91
        from master                        90  
        from using                          1  

    Matched                               855  
    -----------------------------------------


    Result                      Number of obs
    -----------------------------------------
    Not matched                            14
        from master                        14  
        from using                          0  

    Matched                               932  
    -----------------------------------------


    Result                      Number of obs
    -----------------------------------------
    Not matched                           827
        from master          

### Merging Multiple Time Series: A Two-Step Process

We now have five temporary datasets:
1. `temp_gdp.dta` (quarterly)
2. `temp_cpi.dta` (monthly)
3. `temp_interest.dta` (monthly)
4. `temp_unemployment.dta` (monthly)
5. `temp_stock.dta` (monthly)

Our goal: Create **one monthly dataset** with all variables aligned by date.

**The problem:** GDP is quarterly, others are monthly. We need to handle this carefully.

---

**Step 1: Convert Quarterly GDP to Monthly**

**The challenge:**
- GDP: 4 observations per year (2020q1, 2020q2, 2020q3, 2020q4)
- Others: 12 observations per year (2020m1, 2020m2, ..., 2020m12)

**Options for quarterly → monthly conversion:**

1. **Interpolation**: Create intermediate values between quarters
   - Linear: Draw straight line between quarters
   - Cubic spline: Smooth curve through quarters
   - **Pros**: Creates realistic monthly values
   - **Cons**: Introduces artificial variation not in original data

2. **Aggregation markers**: Missing for 2 of 3 months, value in 3rd
   - E.g., Q1 GDP only in March (Jan, Feb missing)
   - **Pros**: Doesn't create fake data
   - **Cons**: Many missing values, VAR can't handle

3. **Carry forward (our choice)**: Repeat quarterly value for all 3 months
   - E.g., 2020q1 GDP → same value for Jan, Feb, March
   - **Pros**: No artificial variation, all months have data
   - **Cons**: Ignores within-quarter dynamics, overstates persistence

**Our implementation:**

```stata
expand 3
```

**The `expand` command:**
- **Syntax**: `expand exp` or `expand varname`
- **What it does**: Duplicates each observation `exp` times
- **Our use**: Each quarterly observation becomes 3 monthly observations
- **Example**:
  - Before: 1 obs for 2020q1 (GDP = 19,000)
  - After: 3 obs for 2020q1 (all with GDP = 19,000)

**Creating the month identifier:**
```stata
bysort date: gen month_in_quarter = _n
```
- `_n` within each quarter: 1, 2, 3
- Identifies which month within the quarter

**Computing the actual month:**
```stata
gen month = (quarter - 1) * 3 + month_in_quarter
```

**Logic:**
- Q1 (quarter=1): months 1, 2, 3 (Jan, Feb, Mar)
  - month = (1-1)*3 + 1 = 1 (Jan)
  - month = (1-1)*3 + 2 = 2 (Feb)
  - month = (1-1)*3 + 3 = 3 (Mar)
- Q2 (quarter=2): months 4, 5, 6 (Apr, May, Jun)
  - month = (2-1)*3 + 1 = 4 (Apr)
  - month = (2-1)*3 + 2 = 5 (May)
  - month = (2-1)*3 + 3 = 6 (Jun)
- Etc.

**Create monthly date:**
```stata
gen date_m = ym(year, month)
```

**Keep one obs per month:**
```stata
bysort date_m: keep if _n == 1
```
- After `expand`, we might have duplicates within a month
- This ensures exactly one observation per month

---

**Step 2: Merge All Monthly Datasets**

**The `merge` Command**

**Syntax:**
```stata
merge join_type varlist using filename [, options]
```

**Join types:**
- **`1:1`**: One-to-one (our case)
  - Each value in master matches at most one in using
  - Each value in using matches at most one in master
  - Perfect for time series: one observation per date
  
- **`1:m`**: One-to-many
  - One master observation matches multiple using observations
  - Example: One country, many years
  
- **`m:1`**: Many-to-one
  - Multiple master observations match one using observation
  - Example: Many individuals in one region
  
- **`m:m`**: Many-to-many (rarely used, discouraged)

**Our merges:**
```stata
merge 1:1 date using "data/processed/temp_interest.dta", nogen
```

**The `nogen` option:**
- By default, `merge` creates `_merge` variable showing merge results:
  - `_merge == 1`: obs in master only (not in using)
  - `_merge == 2`: obs in using only (not in master)
  - `_merge == 3`: obs in both (matched)
- `nogen`: Don't create `_merge` (we're confident about our merges)

**Alternative (checking merge):**
```stata
merge 1:1 date using "temp_interest.dta"
tab _merge  // Check merge results
drop _merge
```

**Sequential merging strategy:**
1. Start with CPI (complete monthly coverage)
2. Merge interest rates (adds `interest_rate` variable)
3. Merge unemployment (adds `unemployment` variable)
4. Merge stock returns (adds `stock_return` variable)
5. Merge GDP (adds `gdp`, `lngdp`, `gdp_growth` variables)

**Why this order?**
- Start with most complete series (CPI)
- Each merge adds variables (columns), not observations (rows)
- Final dataset has dates where ALL series available

**Handling mismatched dates:**
- CPI: 1947-present (most complete)
- Interest rates: 1954-present
- Stock: 1950-present
- GDP: 1947-present
- Unemployment: 1948-present

**Result:** 
- Merged dataset starts when ALL series available (1954-ish)
- Earlier dates (only in CPI) will have missing values for other vars
- Later dates: all variables present

---

**Understanding the Final Dataset**

After merging:
- **Time dimension**: Monthly observations from earliest common date to latest
- **Cross-section**: 7 variables (date, cpi, inflation, interest_rate, unemployment, stock_return, gdp variables)
- **Balanced panel**: All variables observed for all dates (or clearly missing)
- **Ready for VAR**: Same frequency, aligned dates, complete data

In [ ]:
* Add variable labels
label var date "Date (Monthly)"
label var cpi "Consumer Price Index"
label var inflation "Inflation Rate (Monthly % Change)"
label var interest_rate "Interest Rate (%)"
label var unemployment "Unemployment Rate (%)"
label var stock_return "Stock Market Return (Monthly)"
label var gdp "Real GDP (Quarterly, interpolated)"
label var lngdp "Log Real GDP"
label var gdp_growth "GDP Growth Rate (Quarterly)"

* Set time series
tsset date

* Check time series structure
tsset

* Display summary
describe
summarize, detail













Time variable: date, 1947m1 to 2025m11
        Delta: 1 month


Time variable: date, 1947m1 to 2025m11
        Delta: 1 month


Contains data from data/processed/temp_cpi.dta
 Observations:           947                  
    Variables:             9                  5 Nov 2025 01:24
--------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
--------------------------------------------------------------------------------
cpi             float   %9.0g                 Consumer Price Index
date            float   %tm                   Date (Monthly)
inflation       float   %9.0g                 Inflation Rate (Monthly % Change)
interest_rate   float   %9.0g                 Interest Rate (%)
unemployment    float   %9.0g                 Unemployment Rate (%)
stock_return    float   %9.0g                 Stock Market Return (Monthly)
gdp             float   

In [ ]:
* Save processed macro-finance data
save "data/processed/macro_finance_data.dta", replace

display "Macro-finance data processed and saved successfully!"



file data/processed/macro_finance_data.dta saved

Macro-finance data processed and saved successfully!


In [ ]:
* Clean up temporary files
erase "data/processed/temp_gdp.dta"
erase "data/processed/temp_cpi.dta"
erase "data/processed/temp_interest.dta"
erase "data/processed/temp_unemployment.dta"
erase "data/processed/temp_stock.dta"
erase "data/processed/temp_gdp_monthly.dta"

display "Temporary files cleaned up."









Temporary files cleaned up.


### Cleaning Up Temporary Files

**The `erase` Command**

**Syntax:** `erase filename`

**What it does:**
- Permanently deletes a file from disk
- Cannot be undone (no recycle bin/trash in Stata)
- Use with caution!

**Why delete temporary files:**
1. **Organization**: Keep processed folder clean (only final datasets)
2. **Storage**: Free up disk space (though these files are small)
3. **Clarity**: Prevents confusion about which files are final vs. intermediate
4. **Reproducibility**: Temporary files can always be recreated by rerunning code

**Best practices:**
- Only erase files you can regenerate
- Never erase raw data
- Document what gets erased (as we do here)
- Consider keeping temps during development, erase in final version

**Alternative approach:** Save temps in separate folder
```stata
save "data/temp/temp_gdp.dta", replace
```
Then delete entire `temp/` folder when done.

<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">6) Data Quality Checks</h2>

Perform basic quality checks on all processed datasets.


## Why Data Quality Checks Are Essential

After processing data, we must **validate** that everything worked correctly before using it in analysis.

**What can go wrong in data processing:**
1. **Date conversion errors**: Dates parsed incorrectly (e.g., DD/MM/YY vs MM/DD/YY)
2. **Merge failures**: Variables don't align properly across datasets
3. **Transformation errors**: Formulas applied incorrectly
4. **Unexpected missing values**: Data gaps we didn't anticipate
5. **Outliers or data errors**: Values that don't make economic sense
6. **Time series gaps**: Broken temporal structure

**The quality check workflow:**

For each dataset, we check:
1. **Time series structure** (`tsset`): Confirms proper temporal setup
2. **Sample period**: Verify date range matches expectations
3. **Missing values**: Count and understand why they're missing
4. **Distribution**: Summary statistics to spot outliers or errors
5. **Economic plausibility**: Values make sense given what we know about the data

**Benefits:**
- **Catch errors early**: Before they propagate through analysis
- **Document data**: Know sample period, coverage, limitations
- **Build confidence**: Verified data → reliable results
- **Reproducibility**: Others can verify your data processing

In [ ]:
* Check S&P 500 data
use "data/processed/sp500_data.dta", clear
tsset date

display "=== S&P 500 Data Quality Check ==="
display "Date range: " %td r(tmin) " to " %td r(tmax)
display "Number of observations: " _N
count if missing(sp500_close)
count if missing(sp500_return)
summarize sp500_return, detail





Time variable: date, 04nov2015 to 03nov2025, but with gaps
        Delta: 1 day

=== S&P 500 Data Quality Check ===

Date range: 04nov2015 to 03nov2025

Number of observations: 2609

  95

  643


                     S&P 500 Log Returns
-------------------------------------------------------------
      Percentiles      Smallest
 1%    -.0342684      -.0999451
 5%    -.0172119      -.0616093
10%    -.0111666      -.0607519       Obs               1,966
25%     -.003767      -.0532222       Sum of wgt.       1,966

50%     .0006955                      Mean           .0005213
                        Largest       Std. dev.      .0112914
75%     .0058107        .060544
90%     .0117779       .0888085       Variance       .0001275
95%     .0160189       .0896831       Skewness      -.0679821
99%     .0258198       .0908947       Kurtosis       15.01661


### Quality Check: S&P 500 Data

**Understanding `tsset` Output**

When we run `tsset date` (with data already tsset), Stata displays:
```
Time variable: date, 03jan1950 to 31dec2023
        Delta: 1 day
```

**What to verify:**
- **Time variable**: Should be `date` (our standardized name)
- **Range**: Check start and end dates
  - S&P 500 available from 1950s
  - End date should be recent (when data was downloaded)
- **Delta**: Should be 1 day for daily data
- **Format**: Should show `%td` (daily format)

**Using `r()` return values:**

After `tsset`, Stata stores information in return values:
- `r(tmin)`: First time value (numeric)
- `r(tmax)`: Last time value (numeric)
- `r(tdelta)`: Time between observations

**The `display` command with format:**
```stata
display "Date range: " %td r(tmin) " to " %td r(tmax)
```

- `%td`: Formats the numeric date as a readable daily date
- Without `%td`: Would show numbers like 0 (Jan 1, 1960)
- With `%td`: Shows "03jan1950", "31dec2023"

**Checking observation count:**
- `_N`: System variable = total number of observations
- For daily data: approximately 252 trading days/year
- Example: 1950-2023 (73 years) × 252 ≈ 18,400 observations

**Counting missing values:**
```stata
count if missing(sp500_close)
count if missing(sp500_return)
```

**Expected:**
- `sp500_close`: Few missing (maybe 0 if data is complete)
- `sp500_return`: At least 1 missing (first observation has no previous price)

**Additional missing possible:**
- Market closures (9/11 attacks, Hurricane Sandy)
- Data gaps from source
- Should be small number

**Interpreting summary statistics:**

For `sp500_return` (daily log returns):
- **Mean**: Should be small positive (≈ 0.03-0.05% per day → 8-13% annual)
- **Std Dev**: Should be ≈ 1-1.5% (daily volatility)
- **Min**: Extreme crashes (Black Monday 1987: -20%, COVID: -12%)
- **Max**: Extreme rallies (usually < +10%)
- **Skewness**: Slightly negative (crashes bigger than rallies)
- **Kurtosis**: > 3 (fat tails, extreme events more common than normal)

**Red flags:**
- Mean too large (> 0.5% daily): Possible data error
- Std Dev too small (< 0.5%): Missing volatility data
- Returns > 50%: Likely data error or stock split not adjusted
- Too many zeros: Stale prices

In [ ]:
* Check EUR/USD data
use "data/processed/eurusd_data.dta", clear
tsset date

display "=== EUR/USD Data Quality Check ==="
display "Date range: " %td r(tmin) " to " %td r(tmax)
display "Number of observations: " _N
count if missing(eurusd)
count if missing(lneurusd)
count if missing(d_lneurusd)
summarize eurusd lneurusd d_lneurusd, detail





Time variable: date, 04jan1999 to 31oct2025, but with gaps
        Delta: 1 day

=== EUR/USD Data Quality Check ===

Date range: 04jan1999 to 31oct2025

Number of observations: 7000

  270

  270

  1,742


                    EUR/USD Exchange Rate
-------------------------------------------------------------
      Percentiles      Smallest
 1%        .8598           .827
 5%        .9037          .8271
10%       .98485          .8364       Obs               6,730
25%       1.0844          .8364       Sum of wgt.       6,730

50%       1.1702                      Mean           1.182768
                        Largest       Std. dev.      .1540254
75%       1.2994         1.5923
90%      1.37945         1.5924       Variance       .0237238
95%       1.4399         1.5978       Skewness       .1009166
99%       1.5569          1.601       Kurtosis       2.676944

                  Log EUR/USD Exchange Rate
-------------------------------------------------------------
      Percentile

### Quality Check: EUR/USD Data

**What to verify:**

**Time series structure:**
- Delta: 1 day (daily exchange rate)
- Range: EUR introduced January 1, 1999
  - Before 1999: synthetic EUR (weighted basket of predecessor currencies)
  - FRED typically starts 1999-01-04 (first trading day)

**Expected missing values:**

Similar to stock data:
- `eurusd`: Few missing (weekends/holidays)
- `lneurusd`: Same as eurusd (log of non-missing = non-missing)
- `d_lneurusd`: One additional missing (first observation)

**Number of observations:**
- 1999-2023: 24 years × 252 ≈ 6,000 trading days
- Actual slightly less due to holidays

**Interpreting exchange rate statistics:**

**For `eurusd` (level):**
- **Range**: Historically 0.85 to 1.60
  - Low point: ~0.85 (2000-2001, strong USD)
  - High point: ~1.60 (2008, weak USD)
  - Recent: ~1.00-1.20
- **Mean**: Around 1.15-1.20 (over full sample)
- **Non-stationary**: Level has trend/drift

**For `lneurusd` (log level):**
- **Range**: ln(0.85) ≈ -0.16 to ln(1.60) ≈ 0.47
- **Mean**: Around ln(1.15) ≈ 0.14
- Similar non-stationarity as level

**For `d_lneurusd` (daily change / return):**
- **Mean**: Should be ≈ 0 (no persistent trend)
- **Std Dev**: ≈ 0.6-0.8% (daily FX volatility)
  - Lower than stocks (≈1.5%)
  - FX generally less volatile than equities on daily basis
- **Skewness**: Close to 0 (symmetric)
- **Kurtosis**: > 3 (fat tails, FX jumps on news)
- **Min/Max**: Usually ±3-4% (extreme days)
  - Brexit referendum: ~8% move
  - SNB franc de-peg 2015: ~15% move

**Red flags:**
- Level outside 0.8-1.7 range: Likely error
- Daily change > 10%: Check if real event or data error
- Mean change far from 0: Suggests trend (unusual for FX)
- Std dev > 2%: Too volatile for EUR/USD

In [ ]:
* Check Macro-Finance data
use "data/processed/macro_finance_data.dta", clear
tsset date

display "=== Macro-Finance Data Quality Check ==="
display "Date range: " %tm r(tmin) " to " %tm r(tmax)
display "Number of observations: " _N
display "Missing values per variable:"
count if missing(cpi)
display "  CPI: " r(N)
count if missing(inflation)
display "  Inflation: " r(N)
count if missing(interest_rate)
display "  Interest Rate: " r(N)
count if missing(unemployment)
display "  Unemployment: " r(N)
count if missing(stock_return)
display "  Stock Return: " r(N)
count if missing(gdp)
display "  GDP: " r(N)

summarize, detail





Time variable: date, 1947m1 to 2025m11
        Delta: 1 month

=== Macro-Finance Data Quality Check ===

Date range:  1947m1 to 2025m11

Number of observations: 947

Missing values per variable:

  2

  CPI: 2

  3

  Inflation: 3

  91

  Interest Rate: 91

  15

  Unemployment: 15

  826

  Stock Return: 826

  935

  GDP: 935


                    Consumer Price Index
-------------------------------------------------------------
      Percentiles      Smallest
 1%        22.91          21.48
 5%        24.98          21.62
10%        26.85          21.95       Obs                 945
25%        32.75             22       Sum of wgt.         945

50%        109.5                      Mean           123.4215
                        Largest       Std. dev.      88.82391
75%        199.3          321.5
90%      247.284        322.132       Variance       7889.687
95%      276.528        323.364       Skewness       .4530693
99%      317.603        324.368       Kurtosis       1.92712

### Quality Check: Macro-Finance Data

**What to verify:**

**Time series structure:**
- Delta: 1 month (monthly data)
- Format: `%tm` (monthly format like "2020m3")
- Range: Limited by least available series
  - CPI: 1947-present
  - Fed Funds: 1954-present  
  - Others: 1948-present
  - **Effective start**: When ALL series available (≈1954)

**Display format for monthly dates:**
```stata
display "Date range: " %tm r(tmin) " to " %tm r(tmax)
```
- `%tm`: Formats numeric monthly date as "1954m7", "2023m12"

**Expected number of observations:**
- Sample: 1954-2023 → 69 years × 12 months = 828 observations
- Actual may be slightly less if recent months incomplete

---

**Checking Missing Values**

Missing values can occur for several reasons:

**1. Series start at different dates** (most common):
- If we include data from 1947, but Fed Funds starts 1954:
  - 1947m1 to 1954m6: CPI, GDP, etc. present; interest_rate missing
  - After 1954m7: All variables present

**2. Revisions and releases:**
- Most recent months might miss GDP (released with lag)
- Recent CPI might be preliminary

**3. Historical data gaps:**
- WWII period: Some series interrupted
- Methodological changes: Series redefined

**4. Our processing:**
- First observation of growth rates/changes always missing
- But we created these from levels, so minimal impact

**Interpreting the counts:**
```stata
count if missing(cpi)
display "  CPI: " r(N)
```

**Expected patterns:**
- **CPI, inflation**: Should be nearly complete (≈0 missing if 1954+)
- **Interest rates**: Might have a few early or recent missing
- **Unemployment**: Should be complete
- **Stock returns**: First month missing (need previous month for return)
- **GDP, gdp_growth**: 
  - Level (gdp, lngdp): Might miss very recent months
  - Growth: Additionally misses first quarter

---

**Interpreting Summary Statistics**

**Expected ranges for each variable:**

**CPI (level):**
- 1954: ≈27 (base 1982-84=100)
- 2023: ≈300
- Always increasing (except rare deflation)

**Inflation (monthly %):**
- Mean: ≈0.2-0.3% monthly (3-4% annualized)
- Range: -2% to +2% monthly
- Higher in 1970s-80s, lower in 1990s-2000s

**Interest rate (%):**
- Mean: ≈4-5% (over full sample)
- Range: 0% (2008-2015, 2020-2021) to 20% (1980-1981)
- Highly variable across monetary regimes

**Unemployment (%):**
- Mean: ≈5-6%
- Range: 2.5% (1953) to 14.7% (2020)
- Cyclical: rises in recessions

**Stock return (monthly %):**
- Mean: ≈0.8-1.0% monthly (≈10-12% annualized)
- Std Dev: ≈4-5% monthly
- Range: -20% to +15% (extreme months)

**GDP (billions 2017$):**
- 1954: ≈3,000
- 2023: ≈22,000
- Steady growth with business cycles

**GDP growth (quarterly %):**
- Mean: ≈0.6-0.8% quarterly (≈3% annualized)
- Range: -10% (COVID) to +8% (post-recession bouncebacks)
- Note: Carried forward to all 3 months, so appears very persistent

---

**What makes this dataset special:**

This is a **macro-financial VAR dataset** containing:
- **Real economy**: GDP, unemployment (quantity variables)
- **Prices**: Inflation (price level changes)
- **Monetary policy**: Interest rates (policy instrument)
- **Financial markets**: Stock returns (risk asset performance)

**Potential analyses:**
- Impulse responses: How does GDP respond to interest rate shocks?
- Variance decomposition: What drives inflation variation?
- Granger causality: Do stock returns predict GDP?
- Policy evaluation: Effectiveness of monetary policy

<style>
h2.styled-header {
    max-width: 600px;
    margin: 20px auto 22px !important;
    padding: 20px 32px !important;
    border-radius: 18px;
    border: 1px solid #e5e7eb;
    background: linear-gradient(180deg, #ffffff 0%, #f9fafb 100%);
    box-shadow: 0 8px 26px rgba(0,0,0,0.06);
    overflow: hidden;
    text-align: center;
    font-size: 20px !important;
    font-weight: 800 !important;
    color: #14276c !important;
    margin-bottom: 8px !important;
    margin-top: 0 !important;
}
</style>

<h2 class="styled-header">7) Summary and Next Steps</h2>

## What We Accomplished

This notebook has transformed **raw CSV files from FRED** into **analysis-ready Stata datasets**. Here's what we've done:

---

### **1. S&P 500 Data (`sp500_data.dta`)**

**Processing steps:**
- ✓ Imported daily price data
- ✓ Converted string dates to Stata daily format (`%td`)
- ✓ Computed log returns: $r_t = \ln(P_t) - \ln(P_{t-1})$
- ✓ Set time series structure with `tsset`
- ✓ Added descriptive variable labels

**What's ready:**
- Daily returns for **ARMA modeling** (autocorrelation structure)
- Returns for **GARCH models** (volatility clustering)
- Sample for **unit root tests** (random walk hypothesis)

**Key variables:**
- `date`: Daily date (trading days only)
- `sp500_close`: Closing price (level)
- `sp500_return`: Log returns (%)

---

### **2. EUR/USD Data (`eurusd_data.dta`)**

**Processing steps:**
- ✓ Imported daily exchange rate data
- ✓ Converted dates to daily format
- ✓ Created log transformation: $\ln(S_t)$
- ✓ Computed first difference: $\Delta \ln(S_t)$
- ✓ Set time series structure

**What's ready:**
- Levels for **unit root testing** (Dickey-Fuller, Phillips-Perron)
- First differences for **stationarity tests**
- Exchange rate returns for **forecasting models**

**Key variables:**
- `date`: Daily date
- `eurusd`: Exchange rate level (USD per EUR)
- `lneurusd`: Log exchange rate
- `d_lneurusd`: First difference of log rate (≈ returns)

---

### **3. Macro-Finance Data (`macro_finance_data.dta`)**

**Processing steps:**
- ✓ Processed 5 separate economic time series
- ✓ Converted quarterly GDP → monthly (repeated values)
- ✓ Aggregated daily stock returns → monthly
- ✓ Merged all series on monthly dates
- ✓ Created transformations (inflation, GDP growth)
- ✓ Set monthly time series structure

**What's ready:**
- Complete dataset for **Vector Autoregression (VAR)**
- Variables for **impulse response analysis**
- Data for **Granger causality tests**
- Series for **cointegration analysis**

**Key variables:**
- `date`: Monthly date (`%tm` format)
- `cpi`: Consumer Price Index (level)
- `inflation`: Monthly inflation rate (%)
- `interest_rate`: Federal Funds Rate (%)
- `unemployment`: Unemployment rate (%)
- `stock_return`: Monthly S&P 500 return (%)
- `gdp`: Real GDP (billions 2017$, quarterly repeated monthly)
- `lngdp`: Log real GDP
- `gdp_growth`: Quarterly GDP growth rate (%)

---

## Key Stata Skills Learned

**Data Import:**
- `import delimited`: Load CSV files
- `describe`, `list`: Inspect data structure

**Date Handling:**
- `date()`, `ym()`, `yq()`: Parse dates
- `%td`, `%tm`, `%tq`: Date formats
- `year()`, `month()`, `quarter()`: Extract date components

**Time Series Setup:**
- `tsset`: Declare time series structure
- `L.`, `F.`, `D.`: Lag, lead, difference operators

**Data Transformation:**
- `gen`, `egen`: Create variables
- `ln()`: Natural logarithm
- `bysort`: Operations by group
- `expand`: Duplicate observations

**Data Management:**
- `rename`: Change variable names
- `label var`: Add descriptive labels
- `drop`, `keep`: Select variables/observations
- `duplicates`: Handle duplicate observations
- `sort`: Order data

**Merging:**
- `merge 1:1`: One-to-one merge
- Understanding merge types and `_merge` variable

**File Operations:**
- `save`, `use`: Save and load `.dta` files
- `erase`: Delete files

**Validation:**
- `summarize`: Summary statistics
- `count if missing()`: Check missing values
- `_N`, `_n`: Observation counters

---

## Important Concepts Learned

### **Statistical/Econometric:**
1. **Stationarity**: Why we difference and take logs
2. **Returns vs. Prices**: Time series properties differ
3. **Log transformation**: Stabilizes variance, interpretable as %
4. **Integration**: $I(0)$ vs. $I(1)$ processes
5. **Frequency conversion**: Quarterly → Monthly challenges

### **Time Series Theory:**
- Unit roots and random walks
- Autocorrelation in financial data
- Volatility clustering
- Business cycle persistence
- Policy transmission mechanisms

### **Data Management:**
- Raw vs. processed data separation
- Reproducible workflows
- Data validation importance
- Temporary files strategy

---

## Next Steps in Subsequent Notebooks

### **Session 1, Notebook 1: ARMA and Unit Roots**
- Use `sp500_data.dta` and `eurusd_data.dta`
- Estimate ARMA models for returns
- Conduct unit root tests (ADF, PP)
- Test random walk hypothesis

### **Session 1, Notebook 2: VAR and Cointegration**
- Use `macro_finance_data.dta`
- Estimate Vector Autoregression
- Compute impulse response functions
- Test for cointegration (Johansen test)

### **Session 2: Advanced Models**
- GARCH models for volatility (S&P 500)
- State space and Kalman filter (macro data)
- Regime switching models
- Extreme value theory

---

## Best Practices Demonstrated

1. **Always start with `clear all`** - Clean workspace
2. **Document with comments** - Explain what code does
3. **Check data at each step** - `list`, `describe`, `summarize`
4. **Use descriptive variable names** - `sp500_close` not `price`
5. **Add variable labels** - Permanent documentation
6. **Validate after processing** - Quality checks catch errors
7. **Separate raw and processed** - Never modify raw data
8. **Use `tsset` before time series operations** - Required for operators
9. **Handle missing values explicitly** - Use `missing()` function
10. **Keep code reproducible** - Can rerun from raw data

---

## Common Pitfalls to Avoid

**❌ Don't:**
- Use `== .` for missing values (use `missing()` instead)
- Forget `tsset` before using `L.`, `D.`, etc.
- Mix up difference-log and log-difference
- Ignore missing value patterns
- Assume dates parse correctly (always verify!)
- Merge without checking `_merge` results
- Delete raw data files

**✓ Do:**
- Always verify date conversions with `list`
- Check for duplicates before `tsset`
- Use `describe` and `summarize` liberally
- Keep intermediate steps when debugging
- Document units (%, levels, logs)
- Save processed data with clear names

---

## Dataset Availability

All processed datasets are now in `data/processed/`:
- ✓ `sp500_data.dta` - Ready for ARMA/GARCH
- ✓ `eurusd_data.dta` - Ready for unit root tests
- ✓ `macro_finance_data.dta` - Ready for VAR

**You can load them anytime:**
```stata
use "data/processed/sp500_data.dta", clear
use "data/processed/eurusd_data.dta", clear
use "data/processed/macro_finance_data.dta", clear
```

**All datasets are:**
- Properly `tsset`
- Labeled and documented
- Quality-checked
- Ready for econometric analysis

---

## Congratulations!

You've successfully cleaned and prepared three time series datasets using professional workflows. These skills transfer to any time series project:
- Financial data analysis
- Macroeconomic forecasting  
- Policy evaluation
- Academic research

**Now you're ready to start modeling!** 🎉